In [1]:
!pip install azure-ai-ml azure-identity

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import json
import datetime

from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [3]:
# laoding the configuration file
config_file_path = "config.json"

# Read JSON data into a dictionary
config_data = json.load(open(config_file_path))

subscription_id = config_data["subscription_id"]
resource_group = config_data["resource_group"]  
workspace = config_data["workspace"]
location = config_data["location"]

In [4]:

print(resource_group)
print(workspace)
print(location)

deploy-plant
deploy-plant
westus


In [5]:

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [6]:

# Define an endpoint name
endpoint_name = "endpt-plant-dis-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="This is an online endpoint for predicting Plant diseases",
    auth_mode="aml_token"
)

In [7]:
print(endpoint_name)

endpt-plant-dis-03251606792664


In [8]:
model = Model(path="trained_model/plant_disease_prediction_model.h5")
env = Environment(
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_NC4as_T4_v3",
    instance_count=1,
)

In [9]:
%%time
create_endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

CPU times: total: 188 ms
Wall time: 1min 47s


In [10]:
%%time
deploy_endpoint = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpt-plant-dis-03251606792664 exists
Uploading plant_disease_prediction_model.h5 (< 1 MB): 474MB [08:09, 968kB/s]                                           




................................................................................................CPU times: total: 9.89 s
Wall time: 17min 58s


In [11]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
add_traffic = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [12]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://endpt-plant-dis-03251606792664.westus.inference.ml.azure.com/score


In [15]:
import base64
import json

# Read the image  
with open('test_apple_black_rot.jpeg', 'rb') as image_file:
    image_data = image_file.read()

# Convert image to base64
image_base64 = base64.b64encode(image_data).decode('utf-8')

json_request = json.dumps({'data': image_base64})

# Save this JSON as a file
json_file_path = 'sample-request.json'  # Specify your desired path for the JSON file
with open(json_file_path, 'w') as json_file:
    json_file.write(json_request)


# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-request.json",
)

In [16]:
print(response)

"{\"Predicted Class Name\": \"Apple___Black_rot\"}"


In [17]:
# Convert the string to a dictionary
prediction = json.loads(response)

# Now dict_data is a Python dictionary
print(prediction)

{"Predicted Class Name": "Apple___Black_rot"}


In [19]:
print(resource_group)
print(workspace)
print(endpoint_name)

deploy-plant
deploy-plant
endpt-plant-dis-03251606792664


In [20]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

'Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Downloading, Type: Normal, Time: 2025-03-25T10:54:01.551473Z, Message: Start downloading models\nKind: Pod, Name: Pulling, Type: Normal, Time: 2025-03-25T10:54:01.985296Z, Message: Start pulling container image\nKind: Pod, Name: Pulled, Type: Normal, Time: 2025-03-25T10:55:03.047476Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2025-03-25T10:55:03.047476Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2025-03-25T10:55:03.073852Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2025-03-25T10:55:03.325605Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2025-03-25T10:55:21.413935478Z, Message: Container is ready\n\nContainer log

In [23]:
#deleting the endpoint
ml_client.online_endpoints.begin_delete(name=endpoint_name)

...............................................................................................................